In [2]:
from langchain.vectorstores import FAISS
import os 
import pandas as pd
import numpy as np
import re

In [18]:
import json
import re
import nltk
from nltk.corpus import stopwords
stopwords_sp = stopwords.words('spanish')


nombre = 'kicillof'
mes = '10'
filename = 'd:/Facultad/Tesis/Twitter/2019'+mes+'-'+nombre+'.txt'
medios_oficiales = ['Clarín', 'La Nación', 'El Litoral', 'Página|12', 'Bariloche Digital', 'Diario El Ciudadano', 'Radio Mitre', 'infobae', 'TN - Todo Noticias', 'Ámbito Financiero']
medios_oficiales_screnn = ['clarincom', 'lanacion', 'ellitoral', 'pagina12', 'barilochedigital', 'elciudadanoweb', 'radiomitre', 'infobae', 'todonoticias', 'Ambitocom']
medios_oficiales_screnn_2 = ['ellitoral', 'barilochedigital', 'pagina12', 'elciudadanodiario', 'elsigloweb', 'diarionorte', 'diariotextual', 'opisantacruz', 'eldiadelaplata', 'elciudadanoweb', 'chubutparatodos', 'tunoticia', 'diarionoticias', 'lagacetasalta', 'radiomitre', 'elzonda', 'jujuyaldia', 'santacruzalmomento', 'eldoce', 'tncorrientes', 'ultimahora', 'elpregon', 'misionesonline', 'informatesalta', 'losandes', 'laprensa', 'losprimerostv', 'diariouno', 'corrienteshoy', 'elesquiu', 'lamañanaformosa', 'app', 'infobae', 'lagaceta', 'lanacion', 'tn', 'clarin', 'laredlarioja', 'infomerlo', 'm24digital', 'elliberal', 'diariamente', 'chacodiapordia', 'informedigital', 'laprensafederal', 'elindependiente', 'vocescriticas', 'opinionciudadana', 'lamañanacordoba', 'elancasti', 'eltiemposanjuan', 'primeraedicion', 'telam', 'ambito', 'elcomodorense', 'surenio', 'lavoz']
def take_url(json_data):
    try:
        url = json_data['urls'][0]['expanded_url']
    except:
        url = ''
    return url

def replace(x):
    x = str(x)
    x = x.replace('”', '"')
    x = x.replace('“', '"')
    x = x.replace('”', '"')
    x = x.replace('\x93', '"')
    x = x.replace('\x94', '"')
    
def extract_quotes(x):
    return re.findall('"([^"]*)"', x)

def rm_stopwords(frase):
    palabras = nltk.word_tokenize(frase)
    filtered_words = [word for word in palabras if word.lower() not in stopwords_sp]
    return ' '.join(filtered_words)


def borrar_url(tweet, url):
    if url != 0:
        try:
            t = tweet.replace(str(url), "")     
        except:
            t = tweet
    else:
        t = tweet
    return t

def limpiar_tweets(tweet):
    t = re.sub(r'https://t\.co/\S+', '', tweet)
    t = re.sub(r'#\w+', '', t)
    #t = re.sub(r'@\w+', '', t)
    t = t.replace('@','')
    #t = re.sub(r'[^\x00-\x7F]+', '', t)
    emoji_pattern = re.compile("["
                u"\U0001F600-\U0001F64F"  # emoticons
                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                u"\U00002702-\U000027B0"
                u"\U000024C2-\U0001F251"
                "]+", flags=re.UNICODE)
    
    t = emoji_pattern.sub(r'', t)
    t = rm_stopwords(t)
    #t = replace(t)
    t = t.replace("`` ", '"')
    t = t.replace("''", '"')
    return t

def salvando_tweets(filename):
    dias = []
    frases = []
    urls = []
    url_reales = []
    verified = []
    medios = []
    tweets_limpios = []
    with open(filename, 'r', encoding = "utf8") as fp:
        for i, line in enumerate(fp):
            # Para cada linea lee el json y extrae la fecha
            json_data = json.loads(line)
            url_real = take_url(json_data)
            tweet = json_data['text']
            url = json_data['urls']
            extra = json_data
            if tweet[0:2] != 'RT':
                user = json_data['user']
                frases.append(tweet)
                dias.append(json_data['created_at'])
                
                try:
                    verified.append(user['verified'])
                except:
                    verified.append(False)

                try:
                    urls_ = url[0]['url']
                    urls.append(json_data)
                    t = borrar_url(tweet, urls_)
                    url_reales.append(url_real)
                except:
                    t = tweet
                    urls.append(0)
                    url_reales.append(0)


                try:
                    if user['name'] in medios_oficiales or user['screename'] in medios_oficiales_screnn or user['screename'] in medios_oficiales_screnn_2 :
                        medios.append(True)
                    else:
                        medios.append(False)
                except:
                    medios.append(False)

                tweets_limpios.append(limpiar_tweets(tweet))

            if i == 100000:
                break       

        return frases, tweets_limpios, dias, urls, url_reales,medios
    
#frases, tw_limpios,dias, urls, url_reales, medios = salvando_tweets(filename)
frases, tw_limpios, dias, urls, url_reales, medios = salvando_tweets(filename)


In [19]:
df = pd.DataFrame({'tw': tw_limpios, 'fecha':dias, 'id':urls})
len(df)

10021

In [20]:
from sentence_transformers import SentenceTransformer
text = df['tw']
encoder = SentenceTransformer("hiiamsid/sentence_similarity_spanish_es")
vectors = encoder.encode(text)

In [29]:
import faiss

vector_dimension = vectors.shape[1]
index = faiss.IndexFlatL2(vector_dimension)
faiss.normalize_L2(vectors)
index.add(vectors)

In [38]:
faiss.write_index(index, "d:/Facultad/Tesis/large.index")
#index = faiss.read_index("d:/Facultad/Tesis/large.index")


In [70]:
import numpy as np

search_text = 'Hay gente que se dedica a vender droga porque se quedó sin laburo'
search_vector = encoder.encode(search_text)
_vector = np.array([search_vector])
faiss.normalize_L2(_vector)

In [71]:
k = index.ntotal
distances, ann = index.search(_vector, k=k)

In [72]:
ann

array([[5941, 5957, 5967, ..., 3592, 5757,  698]], dtype=int64)

In [73]:
results = pd.DataFrame({'distances': distances[0], 'ann': ann[0]})
results

,distances,ann
0,0.512326,5941
1,0.512326,5957
2,0.512326,5967
3,0.512326,6001
4,0.512326,6054
...,...,...
10016,2.065625,9607
10017,2.071430,8605
10018,2.075234,3592
10019,2.089364,5757


In [77]:
merge = pd.merge(results,df, left_on='ann', right_index=True)
merge[merge['distances']<1]

,distances,ann,tw,fecha,id
0,0.512326,5941,“ gente dedica vender droga quedó laburo ”,Mon Oct 07 09:53:34 +0000 2019,{'created_at': 'Mon Oct 07 09:53:34 +0000 2019...
1,0.512326,5957,“ gente dedica vender droga quedó laburo ”,Mon Oct 07 10:04:04 +0000 2019,{'created_at': 'Mon Oct 07 10:04:04 +0000 2019...
2,0.512326,5967,“ gente dedica vender droga quedó laburo ”,Mon Oct 07 10:09:11 +0000 2019,{'created_at': 'Mon Oct 07 10:09:11 +0000 2019...
3,0.512326,6001,“ gente dedica vender droga quedó laburo ”,Mon Oct 07 10:20:47 +0000 2019,{'created_at': 'Mon Oct 07 10:20:47 +0000 2019...
4,0.512326,6054,“ gente dedica vender droga quedó laburo ”,Mon Oct 07 10:37:12 +0000 2019,{'created_at': 'Mon Oct 07 10:37:12 +0000 2019...
...,...,...,...,...,...
449,0.997255,7293,Most News Now : Axel Kicillof : 'Hay gente ded...,Mon Oct 07 12:43:54 +0000 2019,{'created_at': 'Mon Oct 07 12:43:54 +0000 2019...
450,0.998198,7810,"VIENE SI GANA KICILLOF Axel Kicillof dijo ""gen...",Mon Oct 07 13:33:07 +0000 2019,0
451,0.998336,6814,diferencia tenes elegir si justifican campaña ...,Mon Oct 07 12:09:09 +0000 2019,{'created_at': 'Mon Oct 07 12:09:09 +0000 2019...
452,0.998827,7030,claro Kicillofok volvemos narco provincia . Ju...,Mon Oct 07 12:25:32 +0000 2019,{'created_at': 'Mon Oct 07 12:25:32 +0000 2019...


In [81]:
merge['tw'][451]

'diferencia tenes elegir si justifican campaña narcos imagínate siendo gobierno . VOS ELEGIS SI QUERES VUELVAN NARCOS PROVINCIA Axel Kicillof : "Ahora gente dedica vender droga quedó laburo "'